In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn import metrics
from sklearn import cross_validation

/home/deva/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import matplotlib.pyplot as plt

In [3]:
from ggplot import  *

/home/deva/anaconda2/lib/python2.7/site-packages/ggplot/utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
/home/deva/anaconda2/lib/python2.7/site-packages/ggplot/stats/smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
/home/deva/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
nl_cols = ['DN_0',
'DN_1',
'DN_10',
'DN_11',
'DN_12',
'DN_13',
'DN_14',
'DN_15',
'DN_16',
'DN_17',
'DN_18',
'DN_19',
'DN_2',
'DN_20',
'DN_21',
'DN_22',
'DN_23',
'DN_24',
'DN_25',
'DN_26',
'DN_27',
'DN_28',
'DN_29',
'DN_3',
'DN_30',
'DN_31',
'DN_32',
'DN_33',
'DN_34',
'DN_35',
'DN_36',
'DN_37',
'DN_38',
'DN_39',
'DN_4',
'DN_40',
'DN_41',
'DN_42',
'DN_43',
'DN_44',
'DN_45',
'DN_46',
'DN_47',
'DN_48',
'DN_49',
'DN_5',
'DN_50',
'DN_51',
'DN_52',
'DN_53',
'DN_54',
'DN_55',
'DN_56',
'DN_57',
'DN_58',
'DN_59',
'DN_6',
'DN_60',
'DN_61',
'DN_62',
'DN_63',
'DN_7',
'DN_8',
'DN_9']

In [5]:
modi_var = ['Water',
 'Evergreen Needleleaf forest',
 'Evergreen Broadleaf forest',
 'Deciduous Needleleaf forest',
 'Deciduous Broadleaf forest',
 'Mixed forest',
 'Closed shrublands',
 'Open shrublands',
 'Woody savannas',
 'Savannas',
 'Grasslands',
 'Permanent wetlands',
 'Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'Barren or sparsely vegetated']

In [6]:
labels = ['Village_HHD_Cluster_MSL','Village_HHD_Cluster_MSW','Village_HHD_Cluster_CHH','Village_HHD_Cluster_FC','Village_HHD_Cluster_BF','Village_HHD_Cluster_EMP']

In [9]:
needed_var = ['UID',
'village_code_2011',
'TRU',
'Village_HHD_Cluster_MSL',
'District_HHD_Cluster_MSL',
'Village_HHD_Cluster_MSW',
'District_HHD_Cluster_MSW',
'Village_HHD_Cluster_CHH',
'District_HHD_Cluster_CHH',
'Village_HHD_Cluster_FC',
'District_HHD_Cluster_FC',
'Village_HHD_Cluster_BF',
'District_HHD_Cluster_BF',
'Village_HHD_Cluster_EMP',
'District_HHD_Cluster_EMP',
'Area',
'logMean',
'logSum',
'AvgUrbanNTL',
'CropRatio',
'CropRemainRatio',
'Cropland/Natural vegetation mosaic',
'Croplands',
'Natural',
'UrbanRatio',
'grass_shrubs',
'mod_Cropland/Natural vegetation mosaic',
'mod_Croplands',
'mod_grass_shrubs',
'mod_Urban and built-up',
'MSL_1',
'MSL_2',
'MSL_3',
'MSW_1',
'MSW_2',
'MSW_3',
'CHH_1',
'CHH_2',
'CHH_3',
'CHH_4',
'FC_1',
'FC_2',
'FC_3',
'BF_1',
'BF_2',
'BF_3',
'EMP_1',
'EMP_2',
'EMP_3']

In [10]:
def get_dummies(df_):
    for label in labels:
        df_temp = pd.get_dummies(df_[label])
        df_temp.columns = [label[20:]+'_'+str(i+1) for i in range(len(df_[label].unique()))]
        df_ = pd.concat([df_,df_temp],axis=1)
    return df_

In [11]:
## finding the area:

In [12]:
def find_Area(df_):
    df_['Area'] = df_[nl_cols].sum(axis=1)
    df_ = df_.loc[df_['Area'] > 0]
    return df_

In [13]:
## building features out of modis data

In [14]:
def modis_newFeature(df_):
    df_['Urban and built-up']=df_['Urban and built-up']+1
    df_['CropRatio']=(df_['Croplands']+df_['Cropland/Natural vegetation mosaic'])/df_['Area']
    df_['AvgUrbanNTL']=df_['sum']/df_['Urban and built-up']
    df_['UrbanRatio']= df_['Urban and built-up']/df_['Area']

    df_['Natural']= 0
    for i in ['Water','Evergreen Needleleaf forest','Evergreen Broadleaf forest','Deciduous Needleleaf forest','Deciduous Broadleaf forest','Mixed forest','Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands','Permanent wetlands']:
        df_['Natural']=df_['Natural']+df_[i]

    df_['CropRemainRatio']=(df_['Croplands']+df_['Cropland/Natural vegetation mosaic'])/(df_['Area']-df_['Natural'])
    df_['UrbanRemainRatio']=df_['Urban and built-up']/(df_['Area']-df_['Natural'])
    #df_['UrbanRatio']=df_['UrbanRatio'].fillna(0)
    
    df_['forest'] = df_[modi_var[1:6]].sum(axis=1)
    
    #grass_shrubs consists of shrblands,savansas and grasslands
    df_['grass_shrubs'] = df_[modi_var[6:11]].sum(axis=1)
        
    df_['wet'] = df_[['Water', 'Permanent wetlands']].sum(axis=1)
    for i in modi_var+['Natural','forest','grass_shrubs','wet']:
        df_['mod_'+i]=df_[i]/df_['Area']
    return df_

In [15]:
def to_logSumMean(df_):
    df_['sum'][df_['sum'] < 1]  = 1
    df_['mean'][df_['mean'] < 1] = 1
    df_['logSum'] = np.log(df_['sum'].values)
    df_['logMean'] = np.log(df_['mean'].values)
    return df_

In [8]:
set_names  = ['vill_train_all.csv','vill_cv_all.csv','vill_test_all.csv']

In [17]:
for set_name in set_names:
    print('reading the %s set .....'%(set_name[5:-8]))
    df = pd.read_csv(set_name)
    del df['Unnamed: 0']
    del df['Snow and ice']
    
    print('transforming from multinomial to binomial...')
    df = get_dummies(df)

    print('find the area under that village ... ')
    df = find_Area(df)
    
    print('constructing MODIS new featrues....')
    df = modis_newFeature(df)
    
    print('taking log of sum and mean.....')
    df = to_logSumMean(df)
    
    fname_finalSet = "vill_"+set_name[5:-8]+"_forML.csv"
    print('now saving the needed variabeles in file %s'%fname_finalSet)
    df[needed_var].to_csv(fname_finalSet,index=False)
    print

reading the train set .....
transforming from multinomial to binomial...
find the area under that village ... 
constructing MODIS new featrues....
taking log of sum and mean.....
now saving the needed variabeles in file vill_train_forML.csv


/home/deva/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/deva/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until



reading the cv set .....
transforming from multinomial to binomial...
find the area under that village ... 
constructing MODIS new featrues....
taking log of sum and mean.....
now saving the needed variabeles in file vill_cv_forML.csv

reading the test set .....
transforming from multinomial to binomial...
find the area under that village ... 
constructing MODIS new featrues....
taking log of sum and mean.....
now saving the needed variabeles in file vill_test_forML.csv



In [ ]:
mod_features = ['mean',
 'sum',
 'Water',
 'Permanent wetlands',
 'Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'CropRatio',
 'AvgUrbanNTL',
 'UrbanRatio',
 'Natural',
 'CropRemainRatio',
 'UrbanRemainRatio',
 'forest',
 'grass_shrubs',
 'mod_Water',
 'mod_Permanent wetlands',
 'mod_Croplands',
 'mod_Urban and built-up',
 'mod_Cropland/Natural vegetation mosaic',
 'mod_forest',
 'mod_grass_shrubs',
 'mod_Natural',
 'wet',
 'mod_wet']

In [ ]:
for feature in mod_features:
    plot = ggplot(df_train,aes(x=label,y=feature)) + geom_boxplot()+ylim(df_train[feature].quantile(.025),df_train[feature].quantile(.975))
    fname  = feature.replace("/","_").replace(" ","_") + '_'+label.split("_")[3]+".png"
    plot.save("boxplots/"+fname)

In [ ]:
### now extracting only the needed variables from this dataset to build a classifier....